# Подготовка датасета и обучение YOLO для детекции цифр

1. Структура датасета
Организуйте данные в следующем формате:

Copy
dataset/
├── images/
│   ├── 0_001.png
│   ├── 0_002.png
│   ├── 1_001.png
│   └── ...
└── labels/
    ├── 0_001.txt
    ├── 0_002.txt
    ├── 1_001.txt
    └── ...

2. Разметка данных (YOLO формат)
Для каждого изображения создайте .txt файл с аннотациями в формате:

Copy
<class_id> <x_center> <y_center> <width> <height>

где координаты нормированы на [0, 1] относительно размеров изображения.

Пример labels/0_001.txt:

0 0.5 0.5 0.2 0.3

3. Генерация разметки (автоматическая)
Если цифры занимают все изображение, используйте скрипт:

In [1]:
import os
import cv2

In [2]:
def generate_labels(dataset_root, output_dir):
    """
    Генерирует YOLO-разметку для изображений цифр, где каждый класс в отдельной папке.
    
    :param dataset_root: Путь к корневой папке датасета (содержит папки 0-9)
    :param output_dir: Папка для сохранения файлов разметки
    """
    # Создаем папки для изображений и разметки
    images_dir = os.path.join(output_dir, 'images')
    labels_dir = os.path.join(output_dir, 'labels')
    
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)
    
    # Проходим по всем папкам с цифрами (0-9)
    for class_dir in sorted(os.listdir(dataset_root)):
        class_path = os.path.join(dataset_root, class_dir)
        
        if not os.path.isdir(class_path):
            continue
            
        try:
            class_id = int(class_dir)  # Имя папки - это класс цифры
        except ValueError:
            continue  # Пропускаем нецифровые папки
        
        # Обрабатываем все PNG-файлы в папке класса
        for img_name in os.listdir(class_path):
            if not img_name.endswith('.png'):
                continue
                
            # Полный путь к исходному изображению
            src_img_path = os.path.join(class_path, img_name)
            
            # Новое имя файла (сохраняем структуру "цифра_номер.png")
            new_img_name = f"{class_id}_{img_name.split('_')[1]}"
            dest_img_path = os.path.join(images_dir, new_img_name)
            
            # Копируем изображение в единую папку
            img = cv2.imread(src_img_path)
            cv2.imwrite(dest_img_path, img)
            
            # Генерируем разметку YOLO (всё изображение - одна цифра)
            h, w = img.shape[:2]
            label_content = f"{class_id} 0.5 0.5 1.0 1.0"  # Центр, ширина и высота 100%
            
            # Сохраняем разметку
            label_name = new_img_name.replace('.png', '.txt')
            label_path = os.path.join(labels_dir, label_name)
            
            with open(label_path, 'w') as f:
                f.write(label_content)
    
    print(f"Обработка завершена. Изображения сохранены в {images_dir}, разметка в {labels_dir}")

In [3]:
generate_labels('/home/lastinm/PROJECTS/DATA/digits/images',            # Папка с подпапками 0-9
                 '/home/lastinm/PROJECTS/DATA/digits (yolo format)')    # Куда сохранить подготовленные данные

Обработка завершена. Изображения сохранены в /home/lastinm/PROJECTS/DATA/digits (yolo format)/images, разметка в /home/lastinm/PROJECTS/DATA/digits (yolo format)/labels


## Разделение на train/val

In [4]:
import random
from shutil import copyfile

In [5]:
images_dir = '/home/lastinm/PROJECTS/DATA/digits (yolo format)/images'
labels_dir = '/home/lastinm/PROJECTS/DATA/digits (yolo format)/labels'

In [6]:
# После генерации всех данных:
all_images = os.listdir(images_dir)
random.shuffle(all_images)

# 80% train, 20% val
split_idx = int(0.8 * len(all_images))
train_images = all_images[:split_idx]
val_images = all_images[split_idx:]

# Создаем подпапки
os.makedirs('/home/lastinm/PROJECTS/DATA/digits (yolo format)/train/images', exist_ok=True)
os.makedirs('/home/lastinm/PROJECTS/DATA/digits (yolo format)/train/labels', exist_ok=True)
os.makedirs('/home/lastinm/PROJECTS/DATA/digits (yolo format)/val/images', exist_ok=True)
os.makedirs('/home/lastinm/PROJECTS/DATA/digits (yolo format)/val/labels', exist_ok=True)

# Копируем файлы
for img_name in train_images:
    # Изображения
    src = os.path.join(images_dir, img_name)
    dst = os.path.join('/home/lastinm/PROJECTS/DATA/digits (yolo format)/train/images', img_name)
    copyfile(src, dst)
    
    # Разметка
    label_name = img_name.replace('.png', '.txt')
    src = os.path.join(labels_dir, label_name)
    dst = os.path.join('/home/lastinm/PROJECTS/DATA/digits (yolo format)/train/labels', label_name)
    copyfile(src, dst)

# Аналогично для val...
for img_name in val_images:
    # Изображения
    src = os.path.join(images_dir, img_name)
    dst = os.path.join('/home/lastinm/PROJECTS/DATA/digits (yolo format)/val/images', img_name)
    copyfile(src, dst)
    
    # Разметка
    label_name = img_name.replace('.png', '.txt')
    src = os.path.join(labels_dir, label_name)
    dst = os.path.join('/home/lastinm/PROJECTS/DATA/digits (yolo format)/val/labels', label_name)
    copyfile(src, dst)

# Обучение модели

In [ ]:
%pip install ultralytics

In [3]:
from ultralytics import YOLO

In [ ]:
# Загрузка предобученной модели
model = YOLO('yolov8s.pt')  # small-версия

# Обучение
results = model.train(
    data='/home/lastinm/PROJECTS/DATA/digits_yolo_format/data.yaml',
    epochs=100,
    imgsz=640,  # Модель сама сделает ресайз с сохранением пропорций
    batch=32,
    augment=True,  # Автоматическая аугментация
    hsv_h=0.1,  # Цветовые искажения
    hsv_s=0.7,
    hsv_v=0.4,
    translate=0.2,  # Сдвиги
    scale=0.5,      # Масштабирование
    fliplr=0.5,     # Горизонтальное отражение
    mosaic=1.0,      # Мозаика (особенно полезно для мелких объектов)
#    rect=True,     # Прямоугольное обучение (оптимизирует память)
    device='0'     # GPU
)

New https://pypi.org/project/ultralytics/8.3.112 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 11875MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/lastinm/PROJECTS/DATA/digits_yolo_format/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=Non

RuntimeError: Dataset '/home/lastinm/PROJECTS/DATA/digits_yolo_format/data.yaml' error ❌ '/home/lastinm/PROJECTS/DATA/digits_yolo_format/data.yaml' does not exist

# Inference модели

In [3]:
from ultralytics import YOLO
import cv2
import supervision as sv

In [ ]:
image_path = "artefacts/2_0.93_AgACAgIAAxkBAAICb2gDc1odBGUeYrwLDCqbVKbzKukKAALx7jEb9jsYSCYoYdOUlI6-AQADAgADeAADNgQ.jpeg"

In [9]:
model = YOLO('/home/lastinm/PROJECTS/credit_cards_detection/notebooks/runs/detect/train3/weights/best.pt')
image = cv2.imread(image_path)

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results).with_nms()

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

<Figure size 1200x1200 with 1 Axes>